In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/11/09 19:13:32 WARN Utils: Your hostname, LorenzoMorabito resolves to a loopback address: 127.0.1.1; using 172.24.67.165 instead (on interface eth0)
23/11/09 19:13:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 19:13:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
# read our data - lives in a csv file
path = '/home/unix/git_repo/PySparkExercise/'
file = 'Sample - EU Superstore.csv'
df = spark.read.csv(path + file, header= True, inferSchema= True)

In [6]:
df.show(10)

+------+---------------+----------+----------+--------------+-----------+--------------+---------+-------------+-------+--------------+------+---------------+---------------+------------+--------------------+-------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID| Customer Name|  Segment|         City|  State|       Country|Region|     Product ID|       Category|Sub-Category|        Product Name|  Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+--------------+---------+-------------+-------+--------------+------+---------------+---------------+------------+--------------------+-------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|Aaron Smayling|Corporate|        Leeds|England|United Kingdom| North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes Folders,...|   79.2|       3|     0.0|   39.6|
|     2|ES-2017-1311038|07/0

In [7]:
# how many rows of the EU Superstore dataset have the country being France
df.filter(df['Country'] == 'France').count()

2827

In [8]:
# of those, how many are profitable?
df.filter((df['Country'] == 'France') & (df['Profit'] > 0)).count()

2329

In [9]:
# how any different discount brackets exist? what are they?
df.select('Discount').distinct().count()

14

In [10]:
df.select('Discount').distinct().show()

+--------+
|Discount|
+--------+
|     0.0|
|     0.2|
|     0.7|
|     0.1|
|    0.45|
|     0.6|
|    0.35|
|     0.5|
|    0.85|
|    0.65|
|     0.8|
|     0.4|
|    0.15|
|     0.3|
+--------+



In [11]:
# let's see the totl profit by discount bracket, make sure they are ordered by 
from pyspark.sql.functions import sum, round
df_discount = df.groupBy('Discount').agg(round(sum('Profit'), 2).alias('Rounded_Profit')).orderBy('Discount')
df_discount.show()

+--------+--------------+
|Discount|Rounded_Profit|
+--------+--------------+
|     0.0|     383806.53|
|     0.1|     126884.03|
|    0.15|      24677.56|
|     0.2|       2189.55|
|     0.3|       -758.42|
|    0.35|      -9122.65|
|     0.4|     -21346.43|
|    0.45|      -1103.19|
|     0.5|     -96632.12|
|     0.6|     -20517.46|
|    0.65|      -6221.97|
|     0.7|      -5496.77|
|     0.8|       -460.28|
|    0.85|      -3068.66|
+--------+--------------+



In [12]:
# what is the value after which we should stop offering discount?
df_discount.orderBy(df_discount['Discount'].desc()).filter(df_discount['Rounded_Profit'] >= 0).show(1)

+--------+--------------+
|Discount|Rounded_Profit|
+--------+--------------+
|     0.2|       2189.55|
+--------+--------------+
only showing top 1 row



In [13]:
# who are the top 5 most profitable customers
df_customers = df.groupBy('Customer Name').agg(round(sum('Profit'), 2).alias('Total Profit')) \
    .orderBy('Total Profit', ascending= False).limit(5)
    
df_customers.show()

+-----------------+------------+
|    Customer Name|Total Profit|
+-----------------+------------+
|     Susan Pistek|     4974.51|
|    Patrick Jones|      3986.0|
|Patrick O'Donnell|      3778.2|
|    Ellis Ballard|     3459.66|
|  Mike Gockenbach|     3144.44|
+-----------------+------------+



In [14]:
# get all the rows belonging to those 5 customer names: 
# hint, you may need the collect method - 
# how many rows are they?
l = []
for item in df_customers.collect():
    l.append(item[0])
    
df.filter(df['Customer Name'].isin(l)).count()

76

In [15]:
# create a new column which is the value of the sale 
# were there not discount applied. Hint: orginal = sales/(1-d)

df = df.withColumn('Original_sale', round(df['Sales']/ (1 - df['Discount']), 2))
df.show(5)


+------+---------------+----------+----------+--------------+-----------+--------------+---------+-----+-------+--------------+------+---------------+---------------+------------+--------------------+------+--------+--------+------+-------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID| Customer Name|  Segment| City|  State|       Country|Region|     Product ID|       Category|Sub-Category|        Product Name| Sales|Quantity|Discount|Profit|Original_sale|
+------+---------------+----------+----------+--------------+-----------+--------------+---------+-----+-------+--------------+------+---------------+---------------+------------+--------------------+------+--------+--------+------+-------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|Aaron Smayling|Corporate|Leeds|England|United Kingdom| North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes Folders,...|  79.2|       3|     0.0|  39.6|         79.2|
|     2|ES-2

In [16]:
# calculate the difference between sales and discount value
df = df.withColumn('Diff_sales', df['Sales'] - df['Original_sale'])
df.show(5)

+------+---------------+----------+----------+--------------+-----------+--------------+---------+-----+-------+--------------+------+---------------+---------------+------------+--------------------+------+--------+--------+------+-------------+----------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID| Customer Name|  Segment| City|  State|       Country|Region|     Product ID|       Category|Sub-Category|        Product Name| Sales|Quantity|Discount|Profit|Original_sale|Diff_sales|
+------+---------------+----------+----------+--------------+-----------+--------------+---------+-----+-------+--------------+------+---------------+---------------+------------+--------------------+------+--------+--------+------+-------------+----------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|Aaron Smayling|Corporate|Leeds|England|United Kingdom| North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes Folders,...|  79.2|       3|     0.0| 

In [17]:
# how much money did we not gain due to the discounts - per discount bracket?
df_loss = df.groupBy('Discount').agg(round(sum('Diff_sales'), 2).alias('Loss')).orderBy('Discount')
df_loss.show()

+--------+----------+
|Discount|      Loss|
+--------+----------+
|     0.0|       0.0|
|     0.1| -84712.45|
|    0.15| -45233.18|
|     0.2| -10653.12|
|     0.3|  -2630.24|
|    0.35|  -29163.1|
|     0.4| -46724.69|
|    0.45|  -2083.44|
|     0.5|-183734.27|
|     0.6| -39644.05|
|    0.65| -12219.66|
|     0.7|  -8534.09|
|     0.8|   -635.66|
|    0.85|  -4515.44|
+--------+----------+



In [18]:
# find the discount bracket which made us not gain the most (dynamically)
from pyspark.sql.functions import min
df_loss.agg(min(df_loss['Loss']).alias('Max_loss')).show()

+----------+
|  Max_loss|
+----------+
|-183734.27|
+----------+



In [19]:
# what would have been the total profit if we removed all orders from that discount group? 
max_profit = df.filter(df['Discount'] != 0.5).agg(round(sum(df['Profit']), 2).alias('Max_profit')).collect()[0]['Max_profit']
max_profit

469461.86

In [20]:
#how much more (or less) profit is that?
total_profit = df.agg(round(sum(df['Profit']), 2).alias('Total_profit')).collect()[0]['Total_profit']
total_profit

372829.74

In [21]:
difference_profit = max_profit - total_profit
difference_profit

96632.12

In [22]:
# create a temporary table for our superstore table in sql
df.createOrReplaceTempView('superstore')

In [23]:
spark.sql("SELECT * FROM superstore LIMIT 5").show()

+------+---------------+----------+----------+--------------+-----------+--------------+---------+-----+-------+--------------+------+---------------+---------------+------------+--------------------+------+--------+--------+------+-------------+----------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID| Customer Name|  Segment| City|  State|       Country|Region|     Product ID|       Category|Sub-Category|        Product Name| Sales|Quantity|Discount|Profit|Original_sale|Diff_sales|
+------+---------------+----------+----------+--------------+-----------+--------------+---------+-----+-------+--------------+------+---------------+---------------+------------+--------------------+------+--------+--------+------+-------------+----------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|Aaron Smayling|Corporate|Leeds|England|United Kingdom| North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes Folders,...|  79.2|       3|     0.0| 

In [32]:
# use an SQL query to count the number of rows
spark.sql("SELECT COUNT(*) as N_rows FROM superstore").show()

+------+
|N_rows|
+------+
| 10000|
+------+



In [39]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)
spark.sql("SELECT Country, ROUND(sum(profit)/ sum(sales), 2) as Profit_ratio FROM superstore GROUP BY Country ORDER BY Profit_ratio DESC").show()

+--------------+------------+
|       Country|Profit_ratio|
+--------------+------------+
|   Switzerland|        0.29|
|       Austria|        0.26|
|        Norway|        0.25|
|       Belgium|        0.24|
|United Kingdom|        0.21|
|       Finland|        0.19|
|         Spain|        0.19|
|       Germany|        0.17|
|        France|        0.13|
|         Italy|        0.07|
|       Ireland|       -0.44|
|       Denmark|        -0.5|
|   Netherlands|       -0.53|
|        Sweden|       -0.57|
|      Portugal|       -0.58|
+--------------+------------+



In [43]:
# is the country with the largest profit ratio, the country with the largest profit?
spark.sql("SELECT Country,  ROUND(SUM(Profit), 2) as Tot_profit FROM superstore GROUP BY Country ORDER BY Tot_profit DESC LIMIT 1").show()


+--------------+----------+
|       Country|Tot_profit|
+--------------+----------+
|United Kingdom| 111900.15|
+--------------+----------+



In [46]:
best_profit_country = spark.sql("SELECT Country, \
                                ROUND(SUM(Profit), 2) as Tot_profit \
                                FROM superstore \
                                GROUP BY Country \
                                ORDER BY Tot_profit \
                                DESC LIMIT 1").first()[0]

best_ratio_counrty = spark.sql('''SELECT Country, 
          ROUND(sum(profit)/ sum(sales), 2) as Profit_ratio 
          FROM superstore 
          GROUP BY Country 
          ORDER BY Profit_ratio DESC''').first()[0]

best_profit_country == best_ratio_counrty

False